In [1]:
import numpy as np
import pandas as pd

# Classification on sanitized IDS data

In [ ]:
def data_cleanup(X,y):
    inf_indices=np.where(X==np.inf)
    nan_indices=np.where(X!=X)
    mask = np.ones(len(X), dtype=bool)
    mask[np.union1d(inf_indices[0],nan_indices[0])] = False
    X_mod=X[mask,:]
    y_mod=y[mask]
    return X_mod,y_mod

## Load and clean data

In [ ]:
Friday_nfdump=pd.read_csv('./data/nfdump-CIC-IDS-2017/Friday_Bot_PortScan_DDoS.csv',delimiter=',')

In [ ]:
Friday_nfdump

In [ ]:
Friday_san_cp=Friday_san.copy()

In [ ]:
labels=Friday_san_cp.loc[:,Friday_san.columns[-1]]

In [ ]:
labels.loc[np.where(labels!='Benign')].unique()

In [ ]:
Friday_san_cp.loc[lambda df: df['Label']=='Brute Force -Web','Label']='BruteForce-Web'

In [ ]:
Friday_san_cp.loc[lambda df: df['Label']=='Brute Force -XSS','Label']='BruteForce-XSS'

In [ ]:
Friday_san_cp.loc[lambda df: df['Label']=='SQL Injection','Label']='SQLInjection'

In [ ]:
Friday_san_cp=Friday_san_cp.drop(np.where(labels=='Label')[0])

In [ ]:
Friday_san_cp.to_csv('/Users/arjunbhagoji/IDS_data/flow_data/Friday_BruteForce-Web_BruteForce-XSS_SQLInjection.csv',sep=',')

In [ ]:
len(Friday_san)

In [ ]:
train_split=0.8

In [ ]:
Friday_nfdump_np=Friday_nfdump.to_numpy()
Friday_nfdump_np_perm=Friday_nfdump_np[np.random.permutation(len(Friday_nfdump_np)),:]

# Removing port from features
X=Friday_nfdump_np_perm[:,1:-2]
y=Friday_nfdump_np_perm[:,-1]

In [ ]:
Friday_san

In [ ]:
X

In [ ]:
y_mod=np.zeros(len(y))
y_mod[np.where(y=='Bot')]=1
y_mod[np.where(y=='PortScan')]=2
y_mod[np.where(y=='DDoS')]=3

In [ ]:
X_clean,y_clean=data_cleanup(X,y_mod)

In [ ]:
train_len=int(train_split*len(X_clean))
# Splitting into train and test
X_train=X_clean[:train_len]
X_test=X_clean[train_len:]

y_train=y_clean[:train_len]
y_test=y_clean[train_len:]

## Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=3, random_state=0)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

In [ ]:
y_test_predict=clf.predict(X_test)

In [ ]:
count_correct=0.0
for i in range(len(X_test)):
    if y_test_predict[i]==y_test[i]:
        count_correct+=1
print(count_correct/len(X_test))

In [ ]:
clf.feature_importances_

# Analyzing flows generated from PCAPs

In [2]:
import glob

## Importing data

### nfdump flows

In [51]:
input_dir = '/Users/arjunbhagoji/IDS_data/IDS_2017/flows/Wednesday/nfcapd.*.csv'

input_data_file_list = glob.glob(input_dir)
li = []
print("reading dataframe")
for filename in input_data_file_list:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)
# df = pd.read_csv(args.input_csv, index_col=None, header=0)
# filter out the last few lines that are aggregate stats and not flow records
df = df[~pd.isna(df["pr"])]

reading dataframe


/Users/arjunbhagoji/pytor3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [52]:
df

,ts,te,td,sa,da,sp,dp,pr,flg,fwd,...,mpls8,mpls9,mpls10,cl,sl,al,ra,eng,exid,tr
0,2017-07-05 11:44:09,2017-07-05 11:44:09,0.047,192.168.10.3,192.168.10.1,61418,53.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
1,2017-07-05 11:44:09,2017-07-05 11:44:09,0.047,192.168.10.16,192.168.10.3,45062,53.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
2,2017-07-05 11:41:32,2017-07-05 11:41:32,0.154,192.168.10.9,192.168.10.50,19106,21.0,TCP,...AP.SF,0.0,...,0-0-0,0-0-0,0-0-0,0.117,0.089,2.226,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
3,2017-07-05 11:41:49,2017-07-05 11:41:50,0.092,192.168.10.19,192.168.10.3,35299,53.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
4,2017-07-05 11:38:46,2017-07-05 11:40:42,116.074,192.168.10.17,54.192.36.97,55608,80.0,TCP,...AP.SF,0.0,...,0-0-0,0-0-0,0-0-0,0.063,31.059,31.128,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388208,2017-07-05 09:13:32,2017-07-05 09:13:45,12.669,172.16.0.1,192.168.10.50,37918,80.0,TCP,...APRSF,0.0,...,0-0-0,0-0-0,0-0-0,0.543,0.044,0.305,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
388209,2017-07-05 09:13:45,2017-07-05 09:13:57,11.867,172.16.0.1,192.168.10.50,38450,80.0,TCP,...AP.SF,0.0,...,0-0-0,0-0-0,0-0-0,0.566,0.010,0.289,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
388210,2017-07-05 09:12:30,2017-07-05 09:12:30,0.000,192.168.10.14,192.168.10.3,54194,53.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
388211,2017-07-05 09:13:21,2017-07-05 09:13:58,36.530,192.168.10.50,172.16.0.1,80,37402.0,TCP,...APRSF,0.0,...,0-0-0,0-0-0,0-0-0,0.541,0.046,0.311,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000


### CIC IDS flows

In [53]:
df2=pd.read_csv('data/CIC-IDS-2017/TrafficLabelling/Wednesday-workingHours.pcap_ISCX.csv',index_col=None,header=0)

In [54]:
ids_flows=[df2]
ids_flow_data=pd.concat(ids_flows,axis=0, ignore_index=True)

In [55]:
ids_flow_data.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

## Analyzing data

### Creating list of ids from nfdump data

In [56]:
df_np=df.to_numpy()

In [57]:
id_list=[]
id_list_flipped=[]
for i in range(len(df_np)):
    curr_id=str(df_np[i,4])+'-'+str(df_np[i,3])+'-'+str(int(df_np[i,6]))+'-'+str(int(df_np[i,5]))
    curr_id_flipped=str(df_np[i,3])+'-'+str(df_np[i,4])+'-'+str(int(df_np[i,5]))+'-'+str(int(df_np[i,6]))
    if df_np[i,7]=='UDP':
        curr_id+='-'+str(17)
        curr_id+='-'+str(17)
    elif df_np[i,7]=='TCP':
        curr_id+='-'+str(6)
        curr_id_flipped+='-'+str(6)
    elif 'ICMP' in df_np[i,7]:
        curr_id+='-'+str(0)
        curr_id_flipped+='-'+str(0)
    id_list.append(curr_id)
    id_list_flipped.append(curr_id_flipped)                                                                       

In [58]:
df_id=pd.DataFrame(id_list,columns=['id'])

### nfdump flow stats

In [59]:
# Protocol
df.loc[:,'pr'].value_counts()

UDP      200081
TCP      187786
ICMP         32
ICMP6         8
Name: pr, dtype: int64

In [60]:
# Set of unique ips
source_ips=df.loc[:,'sa'].to_numpy()
dest_ips=df.loc[:,'da'].to_numpy()
all_ips=np.union1d(source_ips,dest_ips)
unique_ips=np.unique(all_ips)

In [64]:
unique_ips[-20:-1]

array(['::', 'fe80::223:aeff:fe9b:9567', 'fe80::223:aeff:fe9b:adb3',
       'fe80::266e:96ff:fe4a:377a', 'fe80::3109:ba1d:7470:7dd',
       'fe80::3c60:fc97:3f94:ac8f', 'fe80::3e30:c3c4:d72f:ae4c',
       'fe80::6d07:ea13:6f73:ed41', 'fe80::8cc1:7756:5ffc:823f',
       'fe80::baac:6fff:fe36:ba8', 'fe80::cd08:22a7:a40f:3d73',
       'fe80::f470:cd16:3cf6:bd7b', 'ff02::1', 'ff02::1:2', 'ff02::1:3',
       'ff02::1:ff36:ba8', 'ff02::1:ff9b:9567', 'ff02::1:ff9b:adb3',
       'ff02::2'], dtype=object)

#### Filtering out ipv6

In [65]:
ipv6_ips=unique_ips[-20:-1]

In [66]:
ipv6_filtered=df[~(df['sa'].isin(ipv6_ips))]
ipv6_filtered_final=ipv6_filtered[~(df['da'].isin(ipv6_ips))]

<ipython-input-66-2d4af759b11d>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ipv6_filtered_final=ipv6_filtered[~(df['da'].isin(ipv6_ips))]


In [67]:
len(ipv6_filtered_final)

387458

In [68]:
# Total flow time
flow_times_array=df.loc[:,'td'].to_numpy().astype(np.float)

In [69]:
# Total flow time in seconds
np.sum(flow_times_array)

15878025.623999998

In [70]:
# Total flow time of filtered data
flow_times_filtered_array=ipv6_filtered_final.loc[:,'td'].to_numpy().astype(np.float)
np.sum(flow_times_filtered_array)

15857211.067000004

In [71]:
# Total hours of flow
np.sum(flow_times_filtered_array)/3600

4404.780851944445

In [72]:
ipv6_filtered_final.loc[:,'pr'].value_counts()

UDP     199640
TCP     187786
ICMP        32
Name: pr, dtype: int64

In [73]:
ipv6_filtered_final_np=ipv6_filtered.to_numpy()

In [74]:
id_list_filtered=[]
id_list_flipped_filtered=[]
for i in range(len(ipv6_filtered_final_np)):
    curr_id=str(ipv6_filtered_final_np[i,4])+'-'+str(ipv6_filtered_final_np[i,3])+'-'+str(int(ipv6_filtered_final_np[i,6]))+'-'+str(int(ipv6_filtered_final_np[i,5]))
    curr_id_flipped=str(ipv6_filtered_final_np[i,3])+'-'+str(ipv6_filtered_final_np[i,4])+'-'+str(int(ipv6_filtered_final_np[i,5]))+'-'+str(int(ipv6_filtered_final_np[i,6]))
    if ipv6_filtered_final_np[i,7]=='UDP':
        curr_id+='-'+str(17)
        curr_id_flipped+='-'+str(17)
    elif ipv6_filtered_final_np[i,7]=='TCP':
        curr_id+='-'+str(6)
        curr_id_flipped+='-'+str(6)
    elif 'ICMP' in ipv6_filtered_final_np[i,7]:
        curr_id+='-'+str(0)
        curr_id_flipped+='-'+str(0)
    id_list_filtered.append(curr_id)
    id_list_flipped_filtered.append(curr_id_flipped)       

In [75]:
id_list_filtered=np.array(id_list_filtered)

#### Timestamps

In [76]:
timestamps=ipv6_filtered_final.sort_values(by=['ts'])

In [77]:
timestamps

,ts,te,td,sa,da,sp,dp,pr,flg,fwd,...,mpls8,mpls9,mpls10,cl,sl,al,ra,eng,exid,tr
360847,2017-07-05 06:42:42,2017-07-05 06:43:18,36.017,192.168.10.3,192.168.10.17,123,123.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
360663,2017-07-05 06:42:42,2017-07-05 06:43:06,24.316,192.168.10.14,40.83.143.209,49460,443.0,TCP,...AP...,0.0,...,0-0-0,0-0-0,0-0-0,457.933,86.519,780.152,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
349403,2017-07-05 06:42:42,2017-07-05 06:46:03,201.065,192.168.10.19,132.163.4.101,123,123.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
360770,2017-07-05 06:42:42,2017-07-05 06:42:42,0.001,192.168.10.17,192.168.10.3,15221,53.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
360839,2017-07-05 06:42:42,2017-07-05 06:43:47,64.290,192.168.10.17,192.168.10.255,138,138.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255134,2017-07-05 15:08:17,2017-07-05 15:08:17,0.000,192.168.10.50,192.168.10.3,20345,53.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
254606,2017-07-05 15:08:18,2017-07-05 15:08:18,0.000,192.168.10.12,192.168.10.255,138,138.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
254736,2017-07-05 15:08:52,2017-07-05 15:08:53,0.985,209.48.71.129,192.168.10.15,80,58122.0,TCP,...A...F,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
255157,2017-07-05 15:09:52,2017-07-05 15:09:52,0.000,192.168.10.3,192.168.10.255,138,138.0,UDP,........,0.0,...,0-0-0,0-0-0,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000


In [27]:
'192.168.10.50' in ipv6_filtered_final['sa'].unique()

True

#### Malicious source and unique ips

In [107]:
mal_ip_filtered=ipv6_filtered_final[(ipv6_filtered_final['sa']=='172.16.0.1') & (ipv6_filtered_final['da']=='192.168.10.50') | (ipv6_filtered_final['da']=='172.16.0.1') & (ipv6_filtered_final['sa']=='192.168.10.50')]

In [110]:
len(mal_ip_filtered)

82628

In [29]:
mal_ip_filtered.loc[:,'da'].value_counts()

192.168.10.50    6994
192.168.10.51     206
Name: da, dtype: int64

In [102]:
mal_ip_filtered.loc[:,'ts'].value_counts()

2017-07-05 08:43:23    1243
2017-07-05 08:45:04    1211
2017-07-05 08:55:21    1208
2017-07-05 08:43:57    1201
2017-07-05 08:44:31    1186
                       ... 
2017-07-05 08:29:11       1
2017-07-05 07:54:13       1
2017-07-05 08:34:15       1
2017-07-05 08:27:56       1
2017-07-05 07:59:16       1
Name: ts, Length: 1848, dtype: int64

In [108]:
mal_indices_df=mal_ip_filtered[(mal_ip_filtered['ts']>'2017-07-05 07:30:00') & (mal_ip_filtered['ts']<'2017-07-05 13:00:00')].sort_values(by=['ts']).index.values

In [109]:
len(mal_indices_df)

82627

In [31]:
ftp_patator_indices_df

array([ 48371, 268461, 266673, ..., 156920, 156414, 156343])

In [32]:
ipv6_filtered_final.loc[ftp_patator_indices_df,:]

,ts,te,td,sa,da,sp,dp,pr,flg,fwd,...,mpls8,mpls9,mpls10,cl,sl,al,ra,eng,exid,tr
48371,2017-07-04 07:17:52,2017-07-04 07:17:57,4.475,172.16.0.1,192.168.10.50,52108,21.0,TCP,...AP.S.,0.0,...,0-0-0,0-0-0,0-0-0,0.787,0.129,2.276,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
268461,2017-07-04 07:19:20,2017-07-04 07:19:29,9.780,172.16.0.1,192.168.10.50,52120,21.0,TCP,...APRSF,0.0,...,0-0-0,0-0-0,0-0-0,0.503,0.065,13.547,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
266673,2017-07-04 07:19:20,2017-07-04 07:19:29,9.796,172.16.0.1,192.168.10.50,52114,21.0,TCP,...APRSF,0.0,...,0-0-0,0-0-0,0-0-0,0.389,0.131,37.604,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
268274,2017-07-04 07:19:20,2017-07-04 07:19:29,9.773,172.16.0.1,192.168.10.50,52128,21.0,TCP,...APRSF,0.0,...,0-0-0,0-0-0,0-0-0,0.476,0.057,11.509,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
269492,2017-07-04 07:19:20,2017-07-04 07:19:29,9.803,172.16.0.1,192.168.10.50,52126,21.0,TCP,...APRSF,0.0,...,0-0-0,0-0-0,0-0-0,0.518,0.015,11.496,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154450,2017-07-04 08:20:07,2017-07-04 08:20:17,9.678,172.16.0.1,192.168.10.50,60132,21.0,TCP,...AP.SF,0.0,...,0-0-0,0-0-0,0-0-0,0.587,0.084,2.580,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
155740,2017-07-04 08:20:10,2017-07-04 08:20:17,6.543,172.16.0.1,192.168.10.50,60138,21.0,TCP,...AP.SF,0.0,...,0-0-0,0-0-0,0-0-0,0.581,0.114,2.370,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
156920,2017-07-04 08:20:10,2017-07-04 08:20:17,6.513,172.16.0.1,192.168.10.50,60142,21.0,TCP,...AP.SF,0.0,...,0-0-0,0-0-0,0-0-0,0.738,0.094,2.224,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000
156414,2017-07-04 08:20:10,2017-07-04 08:20:17,6.628,172.16.0.1,192.168.10.50,60140,21.0,TCP,...AP.SF,0.0,...,0-0-0,0-0-0,0-0-0,0.458,0.046,108.712,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000


In [33]:
ssh_patator_indices_df=mal_ip_filtered[(mal_ip_filtered['ts']>'2017-07-04 12:08:00') & (mal_ip_filtered['ts']<'2017-07-04 13:11:00') & (mal_ip_filtered['da']=='192.168.10.50')].sort_values(by=['ts']).index.values

In [ ]:
portscan_indices_df

In [ ]:
ipv6_filtered_final.loc[portscan_indices_df,:]

In [ ]:
bot_filtered=ipv6_filtered_final[(ipv6_filtered_final['sa']=='205.174.165.73') | (ipv6_filtered_final['da']=='205.174.165.73')]

In [ ]:
bot_filtered

In [ ]:
bot_indices_df=bot_filtered[(bot_filtered['ts']>'2017-07-07 08:00:00') & (bot_filtered['ts']<'2017-07-07 11:00:00')].sort_values(by=['ts']).index.values

In [ ]:
bot_indices_df

### CIC IDS flow stats

In [78]:
source_ips_ids=ids_flow_data.loc[:,' Source IP'].to_numpy()
dest_ips_ids=ids_flow_data.loc[:,' Destination IP'].to_numpy()
all_ips_ids=np.union1d(source_ips_ids,dest_ips_ids)
unique_ips_ids=np.unique(all_ips_ids)

In [79]:
unique_ips_ids

array(['1.193.219.21', '1.226.51.56', '1.234.58.131', ...,
       '98.158.54.105', '99.192.248.32', '99.224.25.39'], dtype=object)

In [80]:
ids_flow_data.loc[:,' Protocol'].value_counts()

6     489450
17    202923
0        330
Name:  Protocol, dtype: int64

In [81]:
flow_times_ids=ids_flow_data.loc[:,' Flow Duration'].to_numpy().astype(np.float)

In [82]:
# Flow times are in microseconds
np.sum(flow_times_ids)/1e6/3600

5388.0134050130555

#### Labels by source ip 

In [83]:
ids_flow_data.loc[:,' Label'].value_counts()

BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name:  Label, dtype: int64

In [84]:
bot_filtered=ids_flow_data[(ids_flow_data[' Label']=='DoS Hulk') | (ids_flow_data[' Label']=='DoS GoldenEye') | (ids_flow_data[' Label']=='DoS slowloris') | (ids_flow_data[' Label']=='DoS Slowhttptest')]

In [98]:
ip_filtered=ids_flow_data[(ids_flow_data[' Destination IP']=='172.16.0.1') & (ids_flow_data[' Source IP']=='192.168.10.50')] 

In [99]:
len(ip_filtered)

6789

In [91]:
len(bot_filtered)

252661

In [85]:
bot_filtered[' Destination IP'].unique()

array(['192.168.10.50'], dtype=object)

In [86]:
bot_filtered

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
6558,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,5169956,8,6,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
26700,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,229,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
26701,172.16.0.1-192.168.10.50-49632-80-6,172.16.0.1,49632,192.168.10.50,80,6,5/7/2017 2:24,5001928,3,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
26702,172.16.0.1-192.168.10.50-49633-80-6,172.16.0.1,49633,192.168.10.50,80,6,5/7/2017 2:24,5125872,8,7,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
26703,172.16.0.1-192.168.10.50-49633-80-6,172.16.0.1,49633,192.168.10.50,80,6,5/7/2017 2:25,214,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692606,172.16.0.1-192.168.10.50-45746-80-6,172.16.0.1,45746,192.168.10.50,80,6,5/7/2017 11:19,11512204,8,5,...,32,892.0,0.0,892.0,892.0,6507197.0,0.0,6507197.0,6507197.0,DoS GoldenEye
692613,172.16.0.1-192.168.10.50-45744-80-6,172.16.0.1,45744,192.168.10.50,80,6,5/7/2017 11:19,11513325,5,5,...,32,918.0,0.0,918.0,918.0,6508582.0,0.0,6508582.0,6508582.0,DoS GoldenEye
692652,172.16.0.1-192.168.10.50-45762-80-6,172.16.0.1,45762,192.168.10.50,80,6,5/7/2017 11:19,11509201,7,6,...,32,899.0,0.0,899.0,899.0,6503248.0,0.0,6503248.0,6503248.0,DoS GoldenEye
692681,172.16.0.1-192.168.10.50-45758-80-6,172.16.0.1,45758,192.168.10.50,80,6,5/7/2017 11:19,11509095,8,5,...,32,914.0,0.0,914.0,914.0,6504954.0,0.0,6504954.0,6504954.0,DoS GoldenEye


In [ ]:
bot_filtered[(bot_filtered[' Source IP']=='205.174.165.73') | (bot_filtered[' Destination IP']=='205.174.165.73')]

In [ ]:
bot_filtered.loc[:,' Destination IP'].value_counts()

In [ ]:
bot_filtered.loc[:,' Source IP'].value_counts()

In [88]:
bot_filtered[(bot_filtered[' Timestamp']<='5/7/2017 9:40')]

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
6558,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,5169956,8,6,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
26700,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,229,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
26701,172.16.0.1-192.168.10.50-49632-80-6,172.16.0.1,49632,192.168.10.50,80,6,5/7/2017 2:24,5001928,3,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
26702,172.16.0.1-192.168.10.50-49633-80-6,172.16.0.1,49633,192.168.10.50,80,6,5/7/2017 2:24,5125872,8,7,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
26703,172.16.0.1-192.168.10.50-49633-80-6,172.16.0.1,49633,192.168.10.50,80,6,5/7/2017 2:25,214,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692606,172.16.0.1-192.168.10.50-45746-80-6,172.16.0.1,45746,192.168.10.50,80,6,5/7/2017 11:19,11512204,8,5,...,32,892.0,0.0,892.0,892.0,6507197.0,0.0,6507197.0,6507197.0,DoS GoldenEye
692613,172.16.0.1-192.168.10.50-45744-80-6,172.16.0.1,45744,192.168.10.50,80,6,5/7/2017 11:19,11513325,5,5,...,32,918.0,0.0,918.0,918.0,6508582.0,0.0,6508582.0,6508582.0,DoS GoldenEye
692652,172.16.0.1-192.168.10.50-45762-80-6,172.16.0.1,45762,192.168.10.50,80,6,5/7/2017 11:19,11509201,7,6,...,32,899.0,0.0,899.0,899.0,6503248.0,0.0,6503248.0,6503248.0,DoS GoldenEye
692681,172.16.0.1-192.168.10.50-45758-80-6,172.16.0.1,45758,192.168.10.50,80,6,5/7/2017 11:19,11509095,8,5,...,32,914.0,0.0,914.0,914.0,6504954.0,0.0,6504954.0,6504954.0,DoS GoldenEye


In [87]:
bot_filtered.sort_values(by=[' Timestamp'])

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
53043,172.16.0.1-192.168.10.50-58394-80-6,172.16.0.1,58394,192.168.10.50,80,6,5/7/2017 10:00,105632393,16,3,...,32,4704874.0,6653051.739,9409292.0,456.0,19200000.0,18800000.0,51300000.0,5841236.0,DoS slowloris
52681,172.16.0.1-192.168.10.50-58340-80-6,172.16.0.1,58340,192.168.10.50,80,6,5/7/2017 10:00,105571782,16,3,...,32,4706498.5,6655518.834,9412661.0,336.0,19200000.0,18800000.0,51300000.0,5772902.0,DoS slowloris
52680,172.16.0.1-192.168.10.50-58338-80-6,172.16.0.1,58338,192.168.10.50,80,6,5/7/2017 10:00,102628933,14,3,...,32,3203118.0,4529113.686,6405685.0,551.0,19200000.0,18800000.0,51300000.0,5964809.0,DoS slowloris
52679,172.16.0.1-192.168.10.50-58336-80-6,172.16.0.1,58336,192.168.10.50,80,6,5/7/2017 10:00,102629959,14,3,...,32,3203735.5,4529985.548,6406919.0,552.0,19200000.0,18800000.0,51300000.0,5964821.0,DoS slowloris
52677,172.16.0.1-192.168.10.50-58334-80-6,172.16.0.1,58334,192.168.10.50,80,6,5/7/2017 10:00,105620322,16,3,...,32,4706904.5,6655551.361,9413090.0,719.0,19200000.0,18800000.0,51300000.0,5804952.0,DoS slowloris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50646,172.16.0.1-192.168.10.50-54466-80-6,172.16.0.1,54466,192.168.10.50,80,6,5/7/2017 9:59,105009188,2,2,...,32,37.0,0.000,37.0,37.0,105000000.0,0.0,105000000.0,105000000.0,DoS slowloris
50645,172.16.0.1-192.168.10.50-54458-80-6,172.16.0.1,54458,192.168.10.50,80,6,5/7/2017 9:59,105009169,2,2,...,32,48.0,0.000,48.0,48.0,105000000.0,0.0,105000000.0,105000000.0,DoS slowloris
50644,172.16.0.1-192.168.10.50-54396-80-6,172.16.0.1,54396,192.168.10.50,80,6,5/7/2017 9:59,105009258,2,2,...,32,50.0,0.000,50.0,50.0,105000000.0,0.0,105000000.0,105000000.0,DoS slowloris
50651,172.16.0.1-192.168.10.50-57996-80-6,172.16.0.1,57996,192.168.10.50,80,6,5/7/2017 9:59,3003278,3,0,...,40,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,DoS slowloris


### Finding common Flow ids

In [ ]:
ids_flow_ids=ids_flow_data.loc[:,'Flow ID']

In [ ]:
ids_flow_ids_np=ids_flow_ids.to_numpy()

In [ ]:
ids_flow_ids_np

In [ ]:
id_list_full=np.hstack((id_list,id_list_flipped))

In [ ]:
common_all, common_arr1, common_arr2=np.intersect1d(ids_flow_ids_np,id_list_full,return_indices=True)

In [ ]:
len(common_all)

In [ ]:
len(id_list_full)

In [ ]:
ids_flow_ids_np[~common_arr1]

In [ ]:
some_ip_filtered_df=df[(df['sa']=='192.168.10.1')]
extra_ip_filtered_df=some_ip_filtered_df[(some_ip_filtered_df['sp']=='53')]
extra_ip_filtered_df

In [ ]:
id_list_full[~common_arr2]

In [ ]:
some_ip_filtered=ids_flow_data[(ids_flow_data[' Destination IP']=='192.168.10.1')]
extra_ip_filtered=some_ip_filtered[(some_ip_filtered[' Destination Port']==53)]
extra_ip_filtered_2=extra_ip_filtered[(extra_ip_filtered[' Source Port']==60933)]
extra_ip_filtered_2

# Adding labels to nfump data

In [92]:
len(ipv6_filtered_final)

387458

In [93]:
ipv6_filtered_final['flow_id']=id_list_filtered
ipv6_filtered_final['flow_id_flipped']=id_list_flipped_filtered

In [94]:
ipv6_filtered_final

,ts,te,td,sa,da,sp,dp,pr,flg,fwd,...,mpls10,cl,sl,al,ra,eng,exid,tr,flow_id,flow_id_flipped
0,2017-07-05 11:44:09,2017-07-05 11:44:09,0.047,192.168.10.3,192.168.10.1,61418,53.0,UDP,........,0.0,...,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000,192.168.10.1-192.168.10.3-53-61418-17,192.168.10.3-192.168.10.1-61418-53-17
1,2017-07-05 11:44:09,2017-07-05 11:44:09,0.047,192.168.10.16,192.168.10.3,45062,53.0,UDP,........,0.0,...,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000,192.168.10.3-192.168.10.16-53-45062-17,192.168.10.16-192.168.10.3-45062-53-17
2,2017-07-05 11:41:32,2017-07-05 11:41:32,0.154,192.168.10.9,192.168.10.50,19106,21.0,TCP,...AP.SF,0.0,...,0-0-0,0.117,0.089,2.226,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000,192.168.10.50-192.168.10.9-21-19106-6,192.168.10.9-192.168.10.50-19106-21-6
3,2017-07-05 11:41:49,2017-07-05 11:41:50,0.092,192.168.10.19,192.168.10.3,35299,53.0,UDP,........,0.0,...,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000,192.168.10.3-192.168.10.19-53-35299-17,192.168.10.19-192.168.10.3-35299-53-17
4,2017-07-05 11:38:46,2017-07-05 11:40:42,116.074,192.168.10.17,54.192.36.97,55608,80.0,TCP,...AP.SF,0.0,...,0-0-0,0.063,31.059,31.128,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000,54.192.36.97-192.168.10.17-80-55608-6,192.168.10.17-54.192.36.97-55608-80-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388208,2017-07-05 09:13:32,2017-07-05 09:13:45,12.669,172.16.0.1,192.168.10.50,37918,80.0,TCP,...APRSF,0.0,...,0-0-0,0.543,0.044,0.305,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000,192.168.10.50-172.16.0.1-80-37918-6,172.16.0.1-192.168.10.50-37918-80-6
388209,2017-07-05 09:13:45,2017-07-05 09:13:57,11.867,172.16.0.1,192.168.10.50,38450,80.0,TCP,...AP.SF,0.0,...,0-0-0,0.566,0.010,0.289,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000,192.168.10.50-172.16.0.1-80-38450-6,172.16.0.1-192.168.10.50-38450-80-6
388210,2017-07-05 09:12:30,2017-07-05 09:12:30,0.000,192.168.10.14,192.168.10.3,54194,53.0,UDP,........,0.0,...,0-0-0,0.000,0.000,0.000,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000,192.168.10.3-192.168.10.14-53-54194-17,192.168.10.14-192.168.10.3-54194-53-17
388211,2017-07-05 09:13:21,2017-07-05 09:13:58,36.530,192.168.10.50,172.16.0.1,80,37402.0,TCP,...APRSF,0.0,...,0-0-0,0.541,0.046,0.311,0.0.0.0,0/0,0.0,1969-12-31 18:00:00.000,172.16.0.1-192.168.10.50-37402-80-6,192.168.10.50-172.16.0.1-80-37402-6


In [95]:
ipv6_filtered_final['label']=np.zeros(len(ipv6_filtered_final))

flow_id_list=ids_flow_data.loc[:,'Flow ID'].to_numpy()
for i, row in ipv6_filtered_final.iterrows():
    curr_id=ipv6_filtered_final.loc[i,'flow_id']
    curr_id_flipped=ipv6_filtered_final.loc[i,'flow_id_flipped']
    if curr_id in flow_id_list:
        ids_index=np.where(flow_id_list==curr_id)[0]
        ipv6_filtered_final.loc[i,'label']=ids_flow_data.loc[ids_index,' Label'].to_numpy()[0]
    elif curr_id_flipped in ids_flow_data.loc[:,'Flow ID'].to_numpy():
        ids_index=np.where(flow_id_list==curr_id_flipped)[0]
        ipv6_filtered_final.loc[i,'label']=ids_flow_data.loc[ids_index,' Label'].to_numpy()[0]

In [ ]:
ipv6_filtered_final

In [43]:
ipv6_filtered_final.loc[ftp_patator_indices_df,'label']='FTP-Patator'

In [44]:
ipv6_filtered_final.loc[ssh_patator_indices_df,'label']='SSH-Patator'

In [ ]:
ipv6_filtered_final.loc[bot_indices_df,'label']='Bot'

In [45]:
benign_indices_df=ipv6_filtered_final[(ipv6_filtered_final['label']==0.0)].index.values

In [46]:
ipv6_filtered_final.loc[benign_indices_df,'label']='BENIGN'

In [47]:
ipv6_filtered_final.loc[:,'label'].value_counts()

BENIGN         299847
FTP-Patator      4003
SSH-Patator      2969
Name: label, dtype: int64

In [48]:
numerical_final=ipv6_filtered_final.loc[:,['td','ipkt','ibyt','opkt','obyt','cl','sl','al','label']]

In [49]:
numerical_final

,td,ipkt,ibyt,opkt,obyt,cl,sl,al,label
0,0.442,4.0,176.0,4.0,416.0,0.000,0.000,0.000,BENIGN
1,0.143,2.0,66.0,2.0,234.0,0.000,0.000,0.000,BENIGN
2,0.000,1.0,60.0,1.0,134.0,0.000,0.000,0.000,BENIGN
3,0.024,1.0,46.0,1.0,62.0,0.000,0.000,0.000,BENIGN
4,0.000,2.0,84.0,2.0,168.0,0.000,0.000,0.000,BENIGN
...,...,...,...,...,...,...,...,...,...
307649,5.552,2.0,52.0,4.0,92.0,0.112,36.304,5516.096,BENIGN
307650,5.073,6.0,584.0,5.0,835.0,0.141,23.416,23.648,BENIGN
307651,0.087,4.0,124.0,2.0,240.0,0.000,0.000,0.000,BENIGN
307652,117.160,17.0,927.0,15.0,1044.0,0.146,37.361,1244.571,BENIGN


In [50]:
numerical_final.to_csv('Tuesday.csv')

# Classification on sanitized nfdump data

In [ ]:
numerical_final

In [ ]:
benign_final=numerical_final[numerical_final['label']=='BENIGN']

In [ ]:
bot_final=numerical_final[numerical_final['label']=='Bot']

In [ ]:
ps_final=numerical_final[numerical_final['label']=='PortScan']

In [ ]:
ddos_final=numerical_final[numerical_final['label']=='DDoS']

In [ ]:
bot_final.median(axis=0)

In [ ]:
benign_final.median()

In [ ]:
ps_final.median()

In [ ]:
ddos_final.median()